In [4]:
## load in relevant packages
#import pandas as pd  
import numpy as np
import gpflow

np.random.seed(1)

In [5]:
## create fake data for model
x_train = np.linspace(-3,3,20)
y_train = np.random.exponential(np.sin(x_train)**2)

print(x_train)
print(y_train)
x_train = x_train.reshape(-1,1)
y_train = y_train.reshape(-1,1)

[-3.         -2.68421053 -2.36842105 -2.05263158 -1.73684211 -1.42105263
 -1.10526316 -0.78947368 -0.47368421 -0.15789474  0.15789474  0.47368421
  0.78947368  1.10526316  1.42105263  1.73684211  2.05263158  2.36842105
  2.68421053  3.        ]
[1.07461729e-02 2.48468821e-01 5.57923279e-05 2.82701687e-01
 1.54373849e-01 9.47276169e-02 1.64580355e-01 2.13716147e-01
 1.05179205e-01 1.91355786e-02 1.34336610e-02 2.40526061e-01
 1.15294365e-01 1.68057852e+00 2.71515819e-02 1.07975433e+00
 4.24109001e-01 3.99003236e-01 2.94999286e-02 4.39666712e-03]


In [6]:
## build model
kernel = gpflow.kernels.RBF(1, active_dims=[0], lengthscales=1.0)
## build model
m = gpflow.models.GPR(x_train, y_train, kern=kernel)
m.likelihood.variance = 0.01

## view 
m.as_pandas_table()

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


,class,prior,transform,trainable,shape,fixed_shape,value
GPR/kern/lengthscales,Parameter,None,+ve,True,(),True,1.0
GPR/kern/variance,Parameter,None,+ve,True,(),True,1.0
GPR/likelihood/variance,Parameter,None,+ve,True,(),True,0.01


In [7]:
gpflow.train.ScipyOptimizer().minimize(m)
print(m.as_pandas_table())

Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use tf.cast instead.


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 11.509249
  Number of iterations: 21
  Number of functions evaluations: 24


INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
  Objective function value: 11.509249
  Number of iterations: 21
  Number of functions evaluations: 24


                             class prior transform  trainable shape  \
GPR/kern/lengthscales    Parameter  None       +ve       True    ()   
GPR/kern/variance        Parameter  None       +ve       True    ()   
GPR/likelihood/variance  Parameter  None       +ve       True    ()   

                         fixed_shape               value  
GPR/kern/lengthscales           True  1.0766695513002522  
GPR/kern/variance               True  0.0837263994557618  
GPR/likelihood/variance         True   0.139101802277386  


In [157]:
## create covariance matrix for new sensor locations
x_new = np.array([4,5,6,7,8])
x_new = x_new.reshape(-1,1)

In [162]:
## example: thought exercise
## 5 sensor locations: S = [4,5,6,7,8]
## total number of sensors to find: k = 2
## for each iteration (1:k):
    ## for each possible sensor location (y) not in A (S):
        ## calculate: corr(y)^2 - corr(yA)*np.linalg.inv(corr(AA))*corr(Ay) / corr(y)^2 - corr(ynA)*np.linalg.inv(corr(nAA))*corr(nAy)
    ## select max(store_val)
    ## append y to A

## possible sensor locations
S = x_new
## number of sensors to find
k = 6
## selected sensors
A = np.array([])

In [159]:
## x is np.array or single value
def generate_cov_mat(x):
    x = np.array(x).reshape(-1,1)
    cov_mat = m.predict_f_full_cov(x)[1][0,:,:]
    return(cov_mat)

In [175]:
def pick_best_sensors_krause(S, k, A):

    ## next version: 
    ## add S + U
    ## add for 2-d, 3-d, etc
    
    ## later version:
    ## use one full matrix and subset from
    
    ## add checks:
    ## k should be less than S
    ## k should be significantly less than S
    ## length of A + k should be less than
    
    for j in range(k):
        ## find coordinates not already in A to iterate through
        Y = S[np.isin(S, A, invert=True)]
        delta = np.array([])
        print("Start iteration: " + str(j))
    
        for y in Y:
            #print(y)
            len_A = A.shape[0]
        
            yA = np.append(y,A)
            nyA = S[np.isin(S, yA, invert=True)]
            len_nA = nyA.shape[0]
            nyA = np.append(y,nyA)
            
            cov_mat_A = generate_cov_mat(yA)
            y_cov = cov_mat_A[:1,[0]]
        
            ## make sure A is not empty, if so: ignore the 2nd part of the calculation
            if len_A > 0:
                AA_cov = np.linalg.inv(cov_mat_A[1:len_A+1, 1:len_A+1])
                yA_cov = cov_mat_A[[0],1:len_A+1]
                Ay_cov = yA_cov.T
                calc_1 = y_cov**2 - np.dot(np.dot(yA_cov, AA_cov), Ay_cov)
            else:
                calc_1 = y_cov**2
            
            cov_mat_nA = generate_cov_mat(nyA)
            nAA_cov = np.linalg.inv(cov_mat_nA[1:len_nA+1, 1:len_nA+1])
            nyA_cov = cov_mat_nA[[0],1:len_nA+1]
            nAy_cov = nyA_cov.T
        
            calc_2 = y_cov**2 - np.dot(np.dot(nyA_cov, nAA_cov), nAy_cov)
            value = calc_1 / calc_2
            delta = np.append(delta, value)
            
        y_star = np.array(Y[np.argmax(delta)])
        A = np.append(A, y_star)
    
    return(A)

In [176]:
best_sensors = pick_best_sensors_krause(S=np.arange(5, 10, 0.1), k=15, A=np.array([]))
print(best_sensors)

Start iteration: 0
Start iteration: 1
Start iteration: 2
Start iteration: 3
Start iteration: 4
Start iteration: 5
Start iteration: 6
Start iteration: 7
Start iteration: 8
Start iteration: 9
Start iteration: 10
Start iteration: 11
Start iteration: 12
Start iteration: 13
Start iteration: 14
[6.6 6.4 6.2 8.  8.2 5.9 6.  9.1 7.3 6.1 6.9 7.9 5.4 5.6 5.3]
